In [2]:
using Plots, DataFrames, OnlineStats, OnlineAI
default(size=(500,300))
df = readtable(joinpath(Pkg.dir("Plots"), "examples", "meetup", "winequality-white.csv"), separator=';');

INFO: Recompiling stale cache file /home/tom/.julia/lib/v0.4/OnlineStats.ji for module OnlineStats.


  0.544174 seconds (2.08 M allocations: 156.080 MB, 2.81% gc time)



maxabs(β - coef(o)) for

glm:      0.006636741266573876
sgd:      NaN
proxgrad: 0.010237129356885588
rda:      1.4318993623506318


  likely near /home/tom/.julia/v0.4/Qwt/src/widgets.jl:5
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:13
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:13
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:13
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:18
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:18
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:21
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:21
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:45
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:120
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:191
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:274
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:336
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:338
  likely near /home/tom/.julia/v0.4/Glob/src/Glob.jl:346


In [4]:
p = plot(10)

(xmeta,ymeta) = (nothing,nothing)

BoundsError: BoundsError: attempt to access 0-element Array{Any,1}
  at index [0]

In [ ]:
p1 = plot(rand(20))
p2 = plot(rand(10))
p3 = scatter(rand(100))
p4 = plot(rand(1000))

In [ ]:
subplot(p1,p2,p3,p4, nr=1, leg=false)

In [ ]:
immerse()
p = plot(rand(10))

In [ ]:
gui()

In [ ]:
append!(p,1,rand(10))
gui()